In [34]:
import glob
import os
import librosa
import numpy as np
import ntpath

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM, Flatten, InputLayer, Conv1D, MaxPooling1D, GRU
from keras.layers import Dropout, BatchNormalization, Bidirectional
from sklearn.model_selection import train_test_split  
from tensorflow.keras.utils import plot_model,to_categorical
import time

import pandas as pd  
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint

import random as rn
import tensorflow as tf
import pandas as pd

In [29]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
#     X = X[0:5645]
    mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40), axis=1)
    return mfcc

In [3]:
def parse_audio_files(filename):
    features = []
    try:
        print('process..', filename)
        extracted = extract_feature(filename)
    except Exception as e:
        print('cannot open', e)
    
    ext_features = np.hstack(extracted)
    features.append(ext_features) 
    return np.array(features)

In [31]:
def create_model(X):  
    model = Sequential()
    model.add(BatchNormalization(axis=-1, input_shape=(X.shape[1],X.shape[2]))) 
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
              
    # model compilation
    model.compile(loss='binary_crossentropy', metrics=['accuracy'])  
    return model

In [57]:
def predict(model, X, name, data):
    classes = model.predict(X)
    print(classes)
    selisih_to_0 = abs(0-classes[0][0])
    selisih_to_1 = abs(1-classes[0][0])
    if(min(selisih_to_0, selisih_to_1) == selisih_to_0):
        dataPd = dataPd.append({'Nama Data':name, 'status':'negatif'}, ignore_index = True)
        print("negatif")
    else:
        dataPd = dataPd.append({'Nama Data':name, 'status':'positif'}, ignore_index = True)
        print("positif")
    data.head()

In [58]:
path_data = 'D:\Mata Kuliah\Tesis\Data\dataweb'
dataPd = pd.DataFrame()
for data in os.listdir(path_data):
    try :
        X = parse_audio_files(os.path.join(path_data, data))
        X = X.reshape((X.shape[0], 1, X.shape[1]))
        model = create_model(X)
        model.load_weights('./model_compare2_lstm_mfccmean.h5')
        predict(model, X, data, dataPd)
    except Exception as e:
        continue

process.. D:\Mata Kuliah\Tesis\Data\dataweb\001de355bf886c7c3ce742ceb4b9afc6.wav
[[0.99970067]]
process.. D:\Mata Kuliah\Tesis\Data\dataweb\00f0faf90ca7079b0ce88e17d6cca4d3.wav
[[3.0993328e-05]]
process.. D:\Mata Kuliah\Tesis\Data\dataweb\05af69ee4f7fc1f99a23a456507bcbbe.wav
[[3.0993328e-05]]
process.. D:\Mata Kuliah\Tesis\Data\dataweb\06652d1a6a6a4b0d2eae1dfc81689995.wav
[[0.00457558]]
process.. D:\Mata Kuliah\Tesis\Data\dataweb\06a68869bdfe3fb016ca2ed6fb18b555.ogg
[[1.2034022e-10]]
process.. D:\Mata Kuliah\Tesis\Data\dataweb\0787c321aee16932771e46b63fff4cdf.wav
[[0.00064698]]
process.. D:\Mata Kuliah\Tesis\Data\dataweb\098dbe285e92780bde13cd6a5b133069.mp3


KeyboardInterrupt: 

In [54]:
dataPd.head()

,Nama Data,status
0,a,positif
